<a href="https://colab.research.google.com/github/diegompin/mtsa/blob/feature%2Fv0.0.8/examples/MTSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MTSA - **M**ultiple **T**ime **S**eries **A**nalysis


### Installing MTSA module:

In [ ]:
!pip install mtsa

### Cloning MTSA repository:

*Obs: This step is necessary to access the example data.*

In [ ]:
!git clone https://github.com/diegompin/mtsa.git

### Setting data directory:

In [2]:
import os
path_input_1 = os.path.join(os.getcwd(),  "sample_data", "machine_type_1", "id_00")
path_input_2 = os.path.join(os.getcwd(),  "mtsa", "examples", "sample_data", "machine_type_1", "id_00")

In [2]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

2024-03-26 21:55:43.148277: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-26 21:55:43.758453: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-03-26 21:55:43.758496: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-03-26 21:55:43.758502: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [3]:
import sys
sys.path.append("..")

### Reading Data Files:

In [4]:
from mtsa import calculate_aucroc

2024-03-30 08:00:22.569548: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:

from mtsa import files_train_test_split
X_train, X_test, y_train, y_test = files_train_test_split(path_input_1)
if(len(y_train) == 0): 
    X_train, X_test, y_train, y_test = files_train_test_split(path_input_2)
y_train

array([1., 1., 1., 1., 1.])

### MFFCMix Model:

In [8]:
from mtsa import MFCCMix
model_mfccmix = MFCCMix()
model_mfccmix.fit(X_train, y_train)
model_mfccmix.model

Pipeline(steps=[('wav2array', Wav2Array()),
                ('array2mfcc', Array2Mfcc(sampling_rate=None)),
                ('features',
                 FeatureUnion(transformer_list=[('M',
                                                 MagnitudeMeanFeatureMfcc()),
                                                ('S',
                                                 MagnitudeStdFeatureMfcc()),
                                                ('C',
                                                 CorrelationFeatureMfcc())])),
                ('final_model', GaussianMixture())])

In [9]:
auc = calculate_aucroc(model_mfccmix, X_test, y_test)
auc

0.52

### Hitachi Model:

In [9]:
from mtsa import Hitachi
model_hitachi = Hitachi()
model_hitachi.fit(X_train, y_train)
model_hitachi.model

Pipeline(steps=[('wav2array', Wav2Array(mono=False)),
                ('demux2array', Demux2Array()),
                ('array2melspec',
                 Array2MelSpec(frames=5, hop_length=512, n_fft=1024, n_mels=64,
                               power=2.0, sampling_rate=None)),
                ('final_model',
                 <mtsa.models.hitachi.AutoEncoderMixin object at 0x7f66601473a0>)])

In [8]:
auc = calculate_aucroc(model_hitachi, X_test, y_test)
auc

14/14 [==============================] - 0s 1ms/step


0.24

### RANSynCoders

#### Data preprocessing

#### Normalization

In [6]:
from mtsa import RANSynCoders

model = RANSynCoders(mono=False)

In [7]:
model.fit(X_train, y_train)

2024-03-30 08:00:34.112787: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-30 08:00:34.113057: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-30 08:00:34.116481: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf


Start of frequency pre-train epoch 0


2024-03-30 08:00:35.262225: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [1102500,8]
	 [[{{node Placeholder/_1}}]]
2024-03-30 08:00:35.262459: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [1102500,8]
	 [[{{node Placeholder/_1}}]]


pre-reconstruction_loss: 0.0026732483
Start of frequency pre-train epoch 1
pre-reconstruction_loss: 0.0022060657
Start of frequency pre-train epoch 2
pre-reconstruction_loss: 0.0020044816
Start of frequency pre-train epoch 3
pre-reconstruction_loss: 0.0023214826
Start of frequency pre-train epoch 4
found common oscillations at period(s) = [43.19971787939344, 21.601128548756833, 38.76445975879892, 9.327332256072284, 19.061203319502077]

Start of sine representation pre-train epoch 0
sine_loss: 0.0044076038
Start of sine representation pre-train epoch 1
sine_loss: 0.004795342
Start of sine representation pre-train epoch 2
sine_loss: 0.0047864085
Start of sine representation pre-train epoch 3
sine_loss: 0.00473958
Start of sine representation pre-train epoch 4
sine_loss: 0.004377666
Start of epoch 0
sine_loss: 0.0044149864 upper_bound_loss: 0.00064771465 lower_bound_loss: 0.0007096799
Start of epoch 1
sine_loss: 0.0051478636 upper_bound_loss: 0.0008361869 lower_bound_loss: 0.0007996675
St

Pipeline(steps=[('wav2array', Wav2Array(mono=False)),
                ('final_model',
                 <mtsa.models.ransyncorders.RANSynCodersBase object at 0x7f9bc614ac40>)])

In [9]:
auc = calculate_aucroc(model, X_test, y_test)
auc

2024-03-30 10:14:49.996799: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [220500,8]
	 [[{{node Placeholder/_1}}]]


TypeError: mid_threshold() got multiple values for argument 'value'

In [14]:
sins, synched, upper, lower = model.predict(X_test)

TypeError: predict() missing 1 required positional argument: 't'